In [4]:
import pandas as pd
import numpy as np
import re
from sklearn.linear_model import LinearRegression
import json


file_path = "prix-et-indices-des-materiaux-construction-national.xlsx"
xls = pd.ExcelFile(file_path)
sheets = xls.sheet_names
all_rows = []


sheets_T1 = [s for s in sheets if "T1" in s.upper()]



for sheet in sheets_T1:

    df_raw = pd.read_excel(xls, sheet_name=sheet, header=None)

    year_row = None
    for i in range(15):
        row = df_raw.iloc[i].astype(str).tolist()
        if sum(1 for c in row if re.match(r"^\s*20\d{2}", c)) >= 5:
            year_row = i
            break

    if year_row is None:
        print("⚠️ Sheet ignorée (pas d'années):", sheet)
        continue

    header_row = year_row - 1
    header_vals = df_raw.iloc[header_row].fillna("").astype(str).tolist()
    year_vals = df_raw.iloc[year_row].fillna("").astype(str).tolist()

    cols = []
    for idx, h in enumerate(header_vals):
        if idx < 4:
            cols.append(h.strip())
        else:
            if idx < len(year_vals) and re.match(r"^\s*20\d{2}", year_vals[idx]):
                cols.append(year_vals[idx].replace(".0", ""))
            else:
                cols.append(h.strip())

    df = pd.read_excel(
        xls,
        sheet_name=sheet,
        header=None,
        skiprows=year_row + 1
    )

    while df.shape[1] < len(cols):
        df[df.shape[1]] = np.nan
    df = df.iloc[:, :len(cols)]

    df.columns = cols

    df.iloc[:, 0] = df.iloc[:, 0].ffill()
    df.iloc[:, 1] = df.iloc[:, 1].ffill()
    df.iloc[:, 2] = df.iloc[:, 2].ffill()
    df.iloc[:, 3] = df.iloc[:, 3].ffill()

    year_cols = [c for c in cols if re.match(r"^\s*20\d{2}", c)]
    meta_cols = [c for c in cols if c not in year_cols]

    df_melt = df.melt(
        id_vars=meta_cols,
        value_vars=year_cols,
        var_name="annees",
        value_name="prix"
    )

    parts = sheet.split()
    region = parts[1] if len(parts) > 1 else "UNKNOWN"
    df_melt["region"] = region

    all_rows.append(df_melt)


df = pd.concat(all_rows, ignore_index=True)

df = df.rename(columns={
    df.columns[2]: "produit",
    df.columns[3]: "variete"
})


def clean_price(x):
    if pd.isna(x):
        return np.nan
    s = str(x).replace(",", ".").replace(" ", "")
    s = re.sub(r"[^0-9\.\-]", "", s)
    try:
        return float(s)
    except:
        return np.nan

df["prix"] = df["prix"].apply(clean_price)
df["prix"] = df["prix"].round(2)


df = df.dropna(subset=["produit", "variete", "prix"])


df.to_csv("clean_data_T1.csv", index=False)
print("✅ CLEAN CSV prêt → clean_data_T1.csv")

df.head(15)


✅ CLEAN CSV prêt → clean_data_T1.csv


,Corps,Activité,produit,variete,annees,prix,region
0,Gros œuvre,Agglomérés et articles en ciment ou en PVC,Agglomérés,15-20/20/40-50 (u),2005,4.52,TTAH
1,Gros œuvre,Agglomérés et articles en ciment ou en PVC,Agglomérés,"7,5-10 /20/40-50 (u)",2005,3.44,TTAH
2,Gros œuvre,Agglomérés et articles en ciment ou en PVC,Hourdis,12-16/20/52 (u),2005,3.28,TTAH
3,Gros œuvre,Agglomérés et articles en ciment ou en PVC,Hourdis,20/20/52 (u),2005,6.41,TTAH
4,Gros œuvre,Agglomérés et articles en ciment ou en PVC,Buse en béton ou PVC,D 100 à 150 mm (ml),2005,21.52,TTAH
5,Gros œuvre,Agglomérés et articles en ciment ou en PVC,Buse en béton ou PVC,D 200 à 300 mm (ml),2005,34.18,TTAH
6,Gros œuvre,Agglomérés et articles en ciment ou en PVC,Bordures,T3 ou T4 (ml),2005,44.03,TTAH
7,Gros œuvre,Agglomérés et articles en ciment ou en PVC,Poutres préfabriquées,12-16/20 (ml),2005,23.03,TTAH
8,Gros œuvre,Agglomérés et articles en ciment ou en PVC,Poutres préfabriquées,20/20 (ml),2005,32.24,TTAH
9,Gros œuvre,Agglomérés et articles en ciment ou en PVC,BPE,BPE (m3),2005,835.93,TTAH


In [3]:
df = pd.read_csv("clean_data_T1.csv")


df["year_int"] = pd.to_numeric(df["annees"], errors="coerce")
df = df.dropna(subset=["year_int"])


combos = df[["region", "produit", "variete"]].drop_duplicates()


future_years = np.arange(2020, 2031)

results = []


for _, row in combos.iterrows():
    region  = row["region"]
    produit = row["produit"]
    variete = row["variete"]

    subset = df[
        (df["region"] == region) &
        (df["produit"] == produit) &
        (df["variete"] == variete)
    ]

    if len(subset) < 2:
        continue

    X = subset["year_int"].values.reshape(-1, 1)
    y = subset["prix"].values


    
    model = LinearRegression()
    model.fit(X, y)

    

    future_pred = model.predict(future_years.reshape(-1, 1))

    for year, pred in zip(future_years, future_pred):
        results.append([
            region,
            produit,
            variete,
            int(year),
            round(pred, 2)
        ])

# Fills missing columns using historical values

# Load historical data with correct schema
df_hist = pd.read_csv("clean_data_T1.csv")
df_hist = df_hist.rename(columns={"annees": "annee", "prix": "prix"})

# Create prediction dataframe
df_pred = pd.DataFrame(
    results,
    columns=["region", "produit", "variete", "annee", "prix"]
)

# Ensure df_pred has all columns from df_hist
for col in df_hist.columns:
    if col not in df_pred.columns:
        df_pred[col] = None

# NOW: Fill missing 'Corps' and 'Activité' using last historical values
for col in ["Corps", "Activité"]:
    for (region, produit, variete), group in df_pred.groupby(["region", "produit", "variete"]):
        # Get last historical row for this combination
        hist_match = df_hist[
            (df_hist["region"] == region) &
            (df_hist["produit"] == produit) &
            (df_hist["variete"] == variete)
        ]

        if len(hist_match) > 0:
            value = hist_match[col].iloc[-1]  # last known value
            df_pred.loc[group.index, col] = value  # fill predictions with it

# Reorder columns
df_pred = df_pred[df_hist.columns]

# Save final prediction file
df_pred.to_csv("predictions_2020_2030.csv", index=False)

print("📁 predictions_2020_2030.csv created with historical category values!")

df_pred.head(15)



📁 predictions_2020_2030.csv created with historical category values!


,Corps,Activité,produit,variete,annee,prix,region
0,Gros œuvre,Agglomérés et articles en ciment ou en PVC,Agglomérés,15-20/20/40-50 (u),2020,5.44,TTAH
1,Gros œuvre,Agglomérés et articles en ciment ou en PVC,Agglomérés,15-20/20/40-50 (u),2021,5.50,TTAH
2,Gros œuvre,Agglomérés et articles en ciment ou en PVC,Agglomérés,15-20/20/40-50 (u),2022,5.56,TTAH
3,Gros œuvre,Agglomérés et articles en ciment ou en PVC,Agglomérés,15-20/20/40-50 (u),2023,5.62,TTAH
4,Gros œuvre,Agglomérés et articles en ciment ou en PVC,Agglomérés,15-20/20/40-50 (u),2024,5.68,TTAH
5,Gros œuvre,Agglomérés et articles en ciment ou en PVC,Agglomérés,15-20/20/40-50 (u),2025,5.74,TTAH
6,Gros œuvre,Agglomérés et articles en ciment ou en PVC,Agglomérés,15-20/20/40-50 (u),2026,5.80,TTAH
7,Gros œuvre,Agglomérés et articles en ciment ou en PVC,Agglomérés,15-20/20/40-50 (u),2027,5.86,TTAH
8,Gros œuvre,Agglomérés et articles en ciment ou en PVC,Agglomérés,15-20/20/40-50 (u),2028,5.92,TTAH
9,Gros œuvre,Agglomérés et articles en ciment ou en PVC,Agglomérés,15-20/20/40-50 (u),2029,5.98,TTAH


In [4]:
# Load historique
df_hist = pd.read_csv("clean_data_T1.csv")
df_hist = df_hist.rename(columns={"annees": "annee", "prix": "prix"})

# Load predictions
df_pred = pd.read_csv("predictions_2020_2030.csv")
df_pred = df_pred.rename(columns={"prix_prevu": "prix"})

# Combine
df_full = pd.concat([df_hist, df_pred], ignore_index=True)

# Sort by region, produit, variete, annee
df_full = df_full.sort_values(by=["region", "produit", "variete", "annee"])

# Save final CSV
df_full.to_csv("full_data_2005_2030.csv", index=False)

print("🎉 CSV complet créé : full_data_2005_2030.csv")


🎉 CSV complet créé : full_data_2005_2030.csv


In [5]:
df_full_json = {
    "meta": {
        "source": "data.gov.ma",
        "description": "Material prices 2005-2030"
    },
    "data": df_full.to_dict(orient="records")
}

with open("materials.json", "w") as f:
    json.dump(df_full_json, f, indent=4)

print("materials.json created successfully!")


materials.json created successfully!


In [ ]:
cd C:\\Users\\hassan\\project_API

In [ ]:
uvicorn main:app --reload
streamlit run app.py
python gradio_app.py

http://127.0.0.1:8000/docs
http://127.0.0.1:8000/data


In [ ]:
python taipy_app.py #doesn't work because i need C++
shiny run app.py --reload
marimo run marimo_app.py

In [3]:
jupyter server list


SyntaxError: invalid syntax (255517071.py, line 1)